<a href="https://colab.research.google.com/github/Erike-Simon/CESAR-AED/blob/main/ProcDados_streaming_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CESAR School Recife**

**Disciplina:** *Processamento de Dados em Larga Escala*

**Alunos:** *Erike Simon, José Aparecido*

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=c4995daca21e7f03815044e632a8074987c9b912fc0b16189a5957741b128a49
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


## Sobre os dados

O arquivo CSV contém eventos 'click' ou 'view' no tempo, de usuários em anúncios de determinadas campanhas.

**Descrição das colunas:**  
timestamp,user_id,action,adId,campaignId

**Amostra:**  
2016-09-21 22:11:00,7c74953c-66cc-48bd-9d02-a02bf039cf3f,click,adId_09,campaignId_01  
2016-06-25 18:29:00,676a083e-2f8e-4ff2-9ec2-270f7f9d6033,view,adId_09,campaignId_02  
2016-02-14 19:03:00,77158997-0dfa-48b7-9149-973dc151ef8d,click,adId_02,campaignId_02  
2016-03-26 06:27:00,78aa2467-b502-413b-94e9-04ec8210bd13,click,adId_07,campaignId_03

**Nome da pasta com os arquivos CSV:**  
data/ad_action_assignment

## Sobre as questões

Usar código de Structured Streaming na resposta.

Favor não alterar o código que gera o inputStream para manter o mesmo padrão na resposta.

Mesmo que não consiga terminar alguma questão, favor enviar, porque parte do código pode valer alguma pontuação.
## ----------------------

In [ ]:
import pyspark.sql.functions as F
import pyspark.sql.types as T

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.streaming import StreamingContext

In [ ]:
# Criando um cluster local com 2 workers, 1 cores por worker e 3GB de RAM por worker

spark = SparkSession.builder\
    .master('local-cluster[2, 1, 3072]')\
    .getOrCreate()
spark

In [ ]:
AD_ACTION_CSV_PATH = "drive/MyDrive/Data/ad_action_assignment"

In [ ]:
# Descomente e execute para parar o SparkSession
# spark.stop()

## Foi utilizado o notebook `assignment_split_ad_action_data.ipynb` para criar os dados do streaming escrevendo arquivos CSV na pasta `data/ad_action_assignment`

## 1) Quantos eventos as campanhas geraram nos últimos 10 segundos? Ordene crescente pelo window.start, descrescente pela quantidade de eventos e calcule a cada 3 segundos. (4 pontos)

In [ ]:
inputStream = spark.readStream.csv(
    AD_ACTION_CSV_PATH,
    schema="timestamp TIMESTAMP, \
        user_id STRING, \
        action STRING, \
        adId STRING, \
        campaignId STRING"
)

inputStream = inputStream.groupBy(
      F.window('timestamp', '10 seconds', '3 seconds'),
      'campaignId'
  ).count()\
  .withColumnRenamed("count", "event_count")\
  .sort(F.col("window.start").asc(), F.col("event_count").desc())\

def foreach_batch_function(df, epoch_id):
    print(epoch_id)
    print(df.toPandas().to_string())

query = inputStream\
    .writeStream\
    .outputMode('complete')\
    .foreachBatch(foreach_batch_function)\
    .start()
query.awaitTermination(60)

0
                                        window     campaignId  event_count
0   (2023-09-01 03:30:51, 2023-09-01 03:31:01)  campaignId_02         1428
1   (2023-09-01 03:30:51, 2023-09-01 03:31:01)  campaignId_03         1418
2   (2023-09-01 03:30:51, 2023-09-01 03:31:01)  campaignId_01         1400
3   (2023-09-01 03:30:54, 2023-09-01 03:31:04)  campaignId_03         5923
4   (2023-09-01 03:30:54, 2023-09-01 03:31:04)  campaignId_02         5786
5   (2023-09-01 03:30:54, 2023-09-01 03:31:04)  campaignId_01         5275
6   (2023-09-01 03:30:57, 2023-09-01 03:31:07)  campaignId_02        10326
7   (2023-09-01 03:30:57, 2023-09-01 03:31:07)  campaignId_03        10223
8   (2023-09-01 03:30:57, 2023-09-01 03:31:07)  campaignId_01         9173
9   (2023-09-01 03:31:00, 2023-09-01 03:31:10)  campaignId_02        14900
10  (2023-09-01 03:31:00, 2023-09-01 03:31:10)  campaignId_03        14625
11  (2023-09-01 03:31:00, 2023-09-01 03:31:10)  campaignId_01        12935
12  (2023-09-01 03:31:0

False

In [ ]:
# Stop job
query.stop()

**Resposta**

1.   Configuramos com os seguintes parâmetros referentes a janela de agrupamento dos dados: windowDuration = 10 segundos e o slideDuration = 3 segundos;
2.   Agrupamos pela campanha e contamos as ocorrências;
3.   Fizemos a ordenação de forma crescente pelo window.start e decrescente pela quantidade de eventos.

Dessa forma obtivemos a listagem abaixo, a qual representa a contagem de eventos para cada campanha no intervalo de 10 segundos, considerando a janela de 3 segundos.




```
                                        window     campaignId  event_count
0   (2023-09-01 03:30:51, 2023-09-01 03:31:01)  campaignId_02         1428
1   (2023-09-01 03:30:51, 2023-09-01 03:31:01)  campaignId_03         1418
2   (2023-09-01 03:30:51, 2023-09-01 03:31:01)  campaignId_01         1400
--------------------------------------------------------------------------
3   (2023-09-01 03:30:54, 2023-09-01 03:31:04)  campaignId_03         5923
4   (2023-09-01 03:30:54, 2023-09-01 03:31:04)  campaignId_02         5786
5   (2023-09-01 03:30:54, 2023-09-01 03:31:04)  campaignId_01         5275
--------------------------------------------------------------------------
6   (2023-09-01 03:30:57, 2023-09-01 03:31:07)  campaignId_02        10326
7   (2023-09-01 03:30:57, 2023-09-01 03:31:07)  campaignId_03        10223
8   (2023-09-01 03:30:57, 2023-09-01 03:31:07)  campaignId_01         9173
--------------------------------------------------------------------------
9   (2023-09-01 03:31:00, 2023-09-01 03:31:10)  campaignId_02        14900
10  (2023-09-01 03:31:00, 2023-09-01 03:31:10)  campaignId_03        14625
11  (2023-09-01 03:31:00, 2023-09-01 03:31:10)  campaignId_01        12935
--------------------------------------------------------------------------
12  (2023-09-01 03:31:03, 2023-09-01 03:31:13)  campaignId_02        15203
13  (2023-09-01 03:31:03, 2023-09-01 03:31:13)  campaignId_03        14511
14  (2023-09-01 03:31:03, 2023-09-01 03:31:13)  campaignId_01        12746
--------------------------------------------------------------------------
15  (2023-09-01 03:31:06, 2023-09-01 03:31:16)  campaignId_02        15351
16  (2023-09-01 03:31:06, 2023-09-01 03:31:16)  campaignId_03        14425
17  (2023-09-01 03:31:06, 2023-09-01 03:31:16)  campaignId_01        12681
--------------------------------------------------------------------------
18  (2023-09-01 03:31:09, 2023-09-01 03:31:19)  campaignId_02        15294
19  (2023-09-01 03:31:09, 2023-09-01 03:31:19)  campaignId_03        14411
20  (2023-09-01 03:31:09, 2023-09-01 03:31:19)  campaignId_01        12749
--------------------------------------------------------------------------
21  (2023-09-01 03:31:12, 2023-09-01 03:31:22)  campaignId_02        15272
22  (2023-09-01 03:31:12, 2023-09-01 03:31:22)  campaignId_03        14386
23  (2023-09-01 03:31:12, 2023-09-01 03:31:22)  campaignId_01        12793
--------------------------------------------------------------------------
24  (2023-09-01 03:31:15, 2023-09-01 03:31:25)  campaignId_02        15294
25  (2023-09-01 03:31:15, 2023-09-01 03:31:25)  campaignId_03        14397
26  (2023-09-01 03:31:15, 2023-09-01 03:31:25)  campaignId_01        12759
--------------------------------------------------------------------------
27  (2023-09-01 03:31:18, 2023-09-01 03:31:28)  campaignId_02        13752
28  (2023-09-01 03:31:18, 2023-09-01 03:31:28)  campaignId_03        12988
29  (2023-09-01 03:31:18, 2023-09-01 03:31:28)  campaignId_01        11465
--------------------------------------------------------------------------
30  (2023-09-01 03:31:21, 2023-09-01 03:31:31)  campaignId_02        13638
31  (2023-09-01 03:31:21, 2023-09-01 03:31:31)  campaignId_03        13123
32  (2023-09-01 03:31:21, 2023-09-01 03:31:31)  campaignId_01        11444
--------------------------------------------------------------------------
33  (2023-09-01 03:31:24, 2023-09-01 03:31:34)  campaignId_02        13549
34  (2023-09-01 03:31:24, 2023-09-01 03:31:34)  campaignId_03        13168
35  (2023-09-01 03:31:24, 2023-09-01 03:31:34)  campaignId_01        11488
--------------------------------------------------------------------------
36  (2023-09-01 03:31:27, 2023-09-01 03:31:37)  campaignId_02        15100
37  (2023-09-01 03:31:27, 2023-09-01 03:31:37)  campaignId_03        14736
38  (2023-09-01 03:31:27, 2023-09-01 03:31:37)  campaignId_01        12614
--------------------------------------------------------------------------
39  (2023-09-01 03:31:30, 2023-09-01 03:31:40)  campaignId_02        15401
40  (2023-09-01 03:31:30, 2023-09-01 03:31:40)  campaignId_03        14463
41  (2023-09-01 03:31:30, 2023-09-01 03:31:40)  campaignId_01        12586
--------------------------------------------------------------------------
42  (2023-09-01 03:31:33, 2023-09-01 03:31:43)  campaignId_02        15375
43  (2023-09-01 03:31:33, 2023-09-01 03:31:43)  campaignId_03        14474
44  (2023-09-01 03:31:33, 2023-09-01 03:31:43)  campaignId_01        12601
--------------------------------------------------------------------------
45  (2023-09-01 03:31:36, 2023-09-01 03:31:46)  campaignId_02        15340
46  (2023-09-01 03:31:36, 2023-09-01 03:31:46)  campaignId_03        14383
47  (2023-09-01 03:31:36, 2023-09-01 03:31:46)  campaignId_01        12727
--------------------------------------------------------------------------
48  (2023-09-01 03:31:39, 2023-09-01 03:31:49)  campaignId_02        15166
49  (2023-09-01 03:31:39, 2023-09-01 03:31:49)  campaignId_03        14527
50  (2023-09-01 03:31:39, 2023-09-01 03:31:49)  campaignId_01        12757
--------------------------------------------------------------------------
51  (2023-09-01 03:31:42, 2023-09-01 03:31:52)  campaignId_02        15359
52  (2023-09-01 03:31:42, 2023-09-01 03:31:52)  campaignId_03        14364
53  (2023-09-01 03:31:42, 2023-09-01 03:31:52)  campaignId_01        12727
--------------------------------------------------------------------------
54  (2023-09-01 03:31:45, 2023-09-01 03:31:55)  campaignId_02        15261
55  (2023-09-01 03:31:45, 2023-09-01 03:31:55)  campaignId_03        14392
56  (2023-09-01 03:31:45, 2023-09-01 03:31:55)  campaignId_01        12797
--------------------------------------------------------------------------
57  (2023-09-01 03:31:48, 2023-09-01 03:31:58)  campaignId_02        15282
58  (2023-09-01 03:31:48, 2023-09-01 03:31:58)  campaignId_03        14326
59  (2023-09-01 03:31:48, 2023-09-01 03:31:58)  campaignId_01        12842
--------------------------------------------------------------------------
60  (2023-09-01 03:31:51, 2023-09-01 03:32:01)  campaignId_02        15172
61  (2023-09-01 03:31:51, 2023-09-01 03:32:01)  campaignId_03        14573
62  (2023-09-01 03:31:51, 2023-09-01 03:32:01)  campaignId_01        12705
--------------------------------------------------------------------------
63  (2023-09-01 03:31:54, 2023-09-01 03:32:04)  campaignId_02        10692
64  (2023-09-01 03:31:54, 2023-09-01 03:32:04)  campaignId_03        10221
65  (2023-09-01 03:31:54, 2023-09-01 03:32:04)  campaignId_01         8802
--------------------------------------------------------------------------
66  (2023-09-01 03:31:57, 2023-09-01 03:32:07)  campaignId_02         6178
67  (2023-09-01 03:31:57, 2023-09-01 03:32:07)  campaignId_03         5884
68  (2023-09-01 03:31:57, 2023-09-01 03:32:07)  campaignId_01         4918
--------------------------------------------------------------------------
69  (2023-09-01 03:32:00, 2023-09-01 03:32:10)  campaignId_03         1561
70  (2023-09-01 03:32:00, 2023-09-01 03:32:10)  campaignId_02         1525
71  (2023-09-01 03:32:00, 2023-09-01 03:32:10)  campaignId_01         1159
```



## 2) Quais são os 5 pares de anúncio e campanha que geraram menos clicks no último minuto? Ordene crescente pelo window.start, crescente pela quantidade de clicks e calcule a cada 30 segundos. (3 pontos)

In [ ]:
inputStream = spark.readStream.csv(
    AD_ACTION_CSV_PATH,
    schema="timestamp TIMESTAMP, \
        user_id STRING, \
        action STRING, \
        adId STRING, \
        campaignId STRING"
)

inputStream = inputStream.groupBy(
      F.window('timestamp', '1 minutes', '30 seconds'),
      'adId', 'campaignId', 'action'
  ).count()\
  .where(F.col('action') == 'click')\

def foreach_batch_function(df, epoch_id):
    window = Window.partitionBy('window.start')\
        .orderBy(F.asc('count'))
    df = df.withColumn('rank', F.row_number().over(window))\
        .where(F.col('rank') <= 5)\
        .drop('rank')\
        .orderBy(F.asc('window.start'), F.asc('count'))
    print(epoch_id)
    print(df.toPandas())

query = inputStream\
    .writeStream\
    .outputMode('complete')\
    .foreachBatch(foreach_batch_function)\
    .start()
query.awaitTermination(50)

0
                                        window     adId     campaignId action  count
0   (2023-09-01 03:30:30, 2023-09-01 03:31:30)  adId_03  campaignId_03  click   2433
1   (2023-09-01 03:30:30, 2023-09-01 03:31:30)  adId_10  campaignId_01  click   2460
2   (2023-09-01 03:30:30, 2023-09-01 03:31:30)  adId_04  campaignId_01  click   2462
3   (2023-09-01 03:30:30, 2023-09-01 03:31:30)  adId_02  campaignId_01  click   2505
4   (2023-09-01 03:30:30, 2023-09-01 03:31:30)  adId_07  campaignId_01  click   2518
5   (2023-09-01 03:31:00, 2023-09-01 03:32:00)  adId_04  campaignId_01  click   4950
6   (2023-09-01 03:31:00, 2023-09-01 03:32:00)  adId_02  campaignId_01  click   4995
7   (2023-09-01 03:31:00, 2023-09-01 03:32:00)  adId_10  campaignId_01  click   5010
8   (2023-09-01 03:31:00, 2023-09-01 03:32:00)  adId_01  campaignId_01  click   5076
9   (2023-09-01 03:31:00, 2023-09-01 03:32:00)  adId_03  campaignId_03  click   5109
10  (2023-09-01 03:31:30, 2023-09-01 03:32:30)  adId_04  campai

False

In [ ]:
# Stop job
query.stop()

**Resposta**

1.   Configuramos com os seguintes parâmetros referentes a janela de agrupamento dos dados: windowDuration = 1 minuto e o slideDuration = 30 segundos;
2.   Agrupamos conjuntamente pelo anúncio, campanha e ação;
3.   Fizemos a contagem do grupo;
4.   Fizemos um filtro pelas ações do tipo click;
3.   Fizemos a ordenação de forma crescente pelo window.start e pela quantidade de eventos.


Dessa forma obtivemos a listagem abaixo, a qual representa os cinco pares de anúncio que que tiveram menos clicks no último minuto, sendo calculado a cada 30 segundos:


```
                                        window     adId     campaignId action  count
0   (2023-09-01 03:30:30, 2023-09-01 03:31:30)  adId_03  campaignId_03  click   2433
1   (2023-09-01 03:30:30, 2023-09-01 03:31:30)  adId_10  campaignId_01  click   2460
2   (2023-09-01 03:30:30, 2023-09-01 03:31:30)  adId_04  campaignId_01  click   2462
3   (2023-09-01 03:30:30, 2023-09-01 03:31:30)  adId_02  campaignId_01  click   2505
4   (2023-09-01 03:30:30, 2023-09-01 03:31:30)  adId_07  campaignId_01  click   2518
-------------------------------------------------------------------------------------
5   (2023-09-01 03:31:00, 2023-09-01 03:32:00)  adId_04  campaignId_01  click   4950
6   (2023-09-01 03:31:00, 2023-09-01 03:32:00)  adId_02  campaignId_01  click   4995
7   (2023-09-01 03:31:00, 2023-09-01 03:32:00)  adId_10  campaignId_01  click   5010
8   (2023-09-01 03:31:00, 2023-09-01 03:32:00)  adId_01  campaignId_01  click   5076
9   (2023-09-01 03:31:00, 2023-09-01 03:32:00)  adId_03  campaignId_03  click   5109
-------------------------------------------------------------------------------------
10  (2023-09-01 03:31:30, 2023-09-01 03:32:30)  adId_04  campaignId_01  click   2563
11  (2023-09-01 03:31:30, 2023-09-01 03:32:30)  adId_02  campaignId_01  click   2568
12  (2023-09-01 03:31:30, 2023-09-01 03:32:30)  adId_01  campaignId_01  click   2622
13  (2023-09-01 03:31:30, 2023-09-01 03:32:30)  adId_10  campaignId_01  click   2624
14  (2023-09-01 03:31:30, 2023-09-01 03:32:30)  adId_06  campaignId_01  click   2757
-------------------------------------------------------------------------------------
15  (2023-09-01 03:32:00, 2023-09-01 03:33:00)  adId_07  campaignId_01  click     72
16  (2023-09-01 03:32:00, 2023-09-01 03:33:00)  adId_08  campaignId_03  click     74
17  (2023-09-01 03:32:00, 2023-09-01 03:33:00)  adId_10  campaignId_01  click     74
18  (2023-09-01 03:32:00, 2023-09-01 03:33:00)  adId_04  campaignId_01  click     75
19  (2023-09-01 03:32:00, 2023-09-01 03:33:00)  adId_02  campaignId_01  click     78
```



## 3) Qual é o total acumulado de clicks e o total acumulado de views? Calcule a medida que os dados são recebidos no streaming (3 pontos)

In [ ]:
inputStream = spark.readStream.csv(
    AD_ACTION_CSV_PATH,
    schema="timestamp TIMESTAMP, \
        user_id STRING, \
        action STRING, \
        adId STRING, \
        campaignId STRING"
)

inputStream = inputStream.groupBy('action').count()

def foreach_batch_function(df, epoch_id):
    print(epoch_id)
    print(df.toPandas())

query = inputStream\
    .writeStream\
    .outputMode('complete')\
    .foreachBatch(foreach_batch_function)\
    .start()
query.awaitTermination(35)

0
  action   count
0   view   76408
1  click  178305


False

In [ ]:
# Stop job
query.stop()

**Resposta**

Para solucionar este problema, primeiro agrupamos os dados com base na coluna 'action' e, em seguida, contamos quantas ocorrências existem em cada categoria. É importante destacar que não definimos um período específico para essa contagem, ou seja, consideramos todos os dados disponíveis durante a execução do programa. Com isso, conseguimos determinar que o número total acumulado de views foi de 76.408 e o de clicks foi de 178.305.

## 4) Qual é a porcentagem de usuários que visualizaram e clicaram em um mesmo anúncio e campanha pelo menos uma vez nos últimos 10 segundos? Calcule a cada 10 segundos. (1 ponto extra na nota final)

Exemplo:
20% de 5 usuários clicaram e visualizaram em um mesmo anúncio e campanha pelo menos uma vez. Nesse caso, dos 5 usuários, 1 tem eventos de click e view em pelo menos 1 par de anúncio e campanha. Enquanto que, 4 usuários não apresentam esse padrão.

```
user_id	adId	campaignId	action
U1		A1		C1 	 		click
U1		A1		C1 	 		view
U1		A2		C1 	 		click
U1		A2		C1 	 		view
U2		A1		C1 	 		view
U3		A1		C1 	 		view
U4		A2		C1 	 		click
U5		A1		C1 	 		click
```

In [ ]:
inputStream = spark.readStream.csv(
    AD_ACTION_CSV_PATH,
    schema="timestamp TIMESTAMP, \
        user_id STRING, \
        action STRING, \
        adId STRING, \
        campaignId STRING"
)

# ESCREVA SEU CÓDIGO AQUI
inputStream = inputStream.groupBy(
      F.window('timestamp', '10 seconds', '10 seconds'),
      'user_id', 'adId', 'campaignId', 'action'
).count()

def foreach_batch_function(df, epoch_id):
    # Contar o número de ocorrências de cada combinação user_id, adId e campaignId
    #counted = df.groupBy('window', 'user_id', 'adId', 'campaignId').count()

    # Filtrar os grupos onde tanto a visualização quanto o clique ocorreram
    filtered = df.where(F.col('count') >= 2)\
       .withColumnRenamed('count', 'click_and_view_cont')

    percentage_df = df.join(filtered, on='window', how='inner')\
       .orderBy('window')

    #percentage_df = df.join(filtered, on='window', how='inner')\
    #  .withColumn('percentage', F.col('click_and_view_cont') / F.col('count') * 100)\
    #  .select('window', F.round('percentage', 2).alias('percentage'))\
    #  .orderBy('window')

    print(epoch_id)
    print(percentage_df.limit(20).toPandas().to_string())
    # Garanta que esse novo DataFrame percentage_df tem as colunas window e percentage
    #print(percentage_df.toPandas())

query = inputStream\
    .writeStream\
    .outputMode('complete')\
    .foreachBatch(foreach_batch_function)\
    .start()
query.awaitTermination(60)
#query.awaitTermination(120)

# a resposta final deve considerar 120 segundos

0
                                        window                               user_id     adId     campaignId action  count                               user_id     adId     campaignId action  click_and_view_cont
0   (2023-09-01 03:31:00, 2023-09-01 03:31:10)  3ecdf2e1-0501-4f71-8402-5adacc7da7db  adId_09  campaignId_02  click      5  3ecdf2e1-0501-4f71-8402-5adacc7da7db  adId_09  campaignId_02  click                    5
1   (2023-09-01 03:31:00, 2023-09-01 03:31:10)  3ecdf2e1-0501-4f71-8402-5adacc7da7db  adId_09  campaignId_02  click      5  0324bb3d-e547-4560-862b-e7fc08e7783b  adId_06  campaignId_03  click                    3
2   (2023-09-01 03:31:00, 2023-09-01 03:31:10)  3ecdf2e1-0501-4f71-8402-5adacc7da7db  adId_09  campaignId_02  click      5  76ae3bf7-de96-4f59-abae-37da446a4bea  adId_06  campaignId_02  click                    2
3   (2023-09-01 03:31:00, 2023-09-01 03:31:10)  3ecdf2e1-0501-4f71-8402-5adacc7da7db  adId_09  campaignId_02  click      5  fe12324b-e5e5-420f-85f

False



```
0  (2023-09-01 03:31:20, 2023-09-01 03:31:30)       38205
1  (2023-09-01 03:31:10, 2023-09-01 03:31:20)       42453
2  (2023-09-01 03:31:40, 2023-09-01 03:31:50)       42450
3  (2023-09-01 03:31:30, 2023-09-01 03:31:40)       42450
4  (2023-09-01 03:32:00, 2023-09-01 03:32:10)        4245
5  (2023-09-01 03:31:50, 2023-09-01 03:32:00)       42450
6  (2023-09-01 03:31:00, 2023-09-01 03:31:10)       42460
```



In [ ]:
# Stop job
query.stop()